In [ ]:
from pathlib import Path

file = Path(__name__).resolve().parent.joinpath("test_input.txt")

data = []
with open(file, "r") as f:
    data = f.read()

data = data.split("\n")

In [ ]:
from typing import Callable

# A bit advance but HO HO HOOO nothing can stop me!

class Monkey:
    def __init__(self, description: list[str]) -> None:
        self.items: list[int] = self.get_start_items(description.pop(0))
        self.operation: Callable[[int], int] = self.get_operation(description.pop(0))
        self.test: Callable[[int], bool] = self.get_test(description.pop(0))
        self.true_monkey: int = int(description.pop(0).split()[-1])
        self.false_monkey: int = int(description.pop(0).split()[-1])
        self.inspected_items = 0

    @staticmethod
    def get_start_items(line) -> list[int]:
        return [int(i.strip()) for i in line.split(":")[1].split(",")]

    @staticmethod
    def get_operation(line) -> Callable[[int], int]:
        description = {}
        function = line.split(":")[1].strip()
        exec(f"def operation(old):\n\t{function}\n\treturn new", description)
        return description["operation"]

    @staticmethod
    def get_test(line) -> Callable[[int], bool]:
        _divisible_by = int(line.split()[-1])

        def test(x: int) -> bool:
            nonlocal _divisible_by
            return not bool(x % _divisible_by)

        return test

    def inspect(self) -> tuple[int, int]:
        if not self.items:
            return None, None
        
        item = self.items.pop(0)
        item = self.operation(item)
        # item = int(item / 3)
        next_monkey = self.true_monkey if self.test(item) else self.false_monkey
        
        self.inspected_items +=1
        
        return next_monkey, item


In [ ]:
from typing import Callable
# A bit advance but HO HO HOOO nothing can stop me!

class Monkey:
    def __init__(self, description: list[str]) -> None:
        self.items: list[int] = self.get_start_items(description.pop(0))
        self.operation: Callable[[int], int] = self.get_operation(description.pop(0))
        self.test: Callable[[int], bool] = self.get_test(description.pop(0))
        self.true_monkey: int = int(description.pop(0).split()[-1])
        self.false_monkey: int = int(description.pop(0).split()[-1])
        self.inspected_items = 0

    @staticmethod
    def get_start_items(line) -> list[int]:
        return [[int(i.strip())] for i in line.split(":")[1].split(",")]

    @staticmethod
    def get_operation(line) -> Callable[[int], int]:
        description = {}
        function = line.split(":")[1].strip()
        
        if "+" in function:
            function = f"old.append({function.split()[-1]})"
            exec(f"def operation(old):\n\t{function}\n\treturn old", description)
            return description["operation"]
        
        function = function.split(" ")[-3:]
        function = " ".join(function)
        function = f"new = [{function} for old in data]"
        print(function)
        exec(f"def operation(data):\n\t{function}\n\treturn new", description)
        return description["operation"]    
        

    @staticmethod
    def get_test(line) -> Callable[[int], bool]:
        _divisible_by = int(line.split()[-1])

        def test(x: list[int]) -> bool:
            nonlocal _divisible_by
            result =  sum([i % _divisible_by for i in x])
            
            return not bool(result % _divisible_by)

        return test

    def inspect(self) -> tuple[int, int]:
        if not self.items:
            return None, None
        
        item = self.items.pop(0)
        item = self.operation(item)
        next_monkey = self.true_monkey if self.test(item) else self.false_monkey
        
        self.inspected_items +=1
        
        return next_monkey, item

In [60]:
monkeys: dict[int, Monkey] = {}
description = []
for idx in range(0,len(data), 7):
    monkey = int(data[idx].split()[-1].strip(":"))
    monkeys[monkey] = Monkey(data[idx+1:idx+6])

new = [old * 19 for old in data]
new = [old * old for old in data]


In [61]:
for round in range(21):
    result = {k:m.inspected_items for k, m in monkeys.items()}
    result2 = {}
    for k, m in monkeys.items():
        result2[k] = [sum(i) for i in m.items]
        # result2[k] = m.items  
    
    print(round, result, result2)
    for monkey in range(len(monkeys)):
        while True:
            next_monkey, item = monkeys[monkey].inspect()
            
            if next_monkey is None:
                break
            
            monkeys[next_monkey].items.append(item)

0 {0: 0, 1: 0, 2: 0, 3: 0} {0: [79, 98], 1: [54, 65, 75, 74], 2: [79, 60, 97], 3: [74]}
1 {0: 2, 1: 4, 2: 3, 3: 6} {0: [60, 71, 81, 80], 1: [77, 1504, 1865, 6244, 3603, 9412], 2: [], 3: []}
2 {0: 6, 1: 10, 2: 3, 3: 10} {0: [83, 1510, 1871, 6250, 3609, 9418], 1: [1143, 1352, 1542, 1523], 2: [], 3: []}
3 {0: 12, 1: 14, 2: 3, 3: 16} {0: [1149, 1358, 1548, 1529], 1: [1580, 28693, 35552, 118753, 68574, 178945], 2: [], 3: []}
4 {0: 16, 1: 20, 2: 3, 3: 20} {0: [1586, 28699, 35558, 118759, 68580, 178951], 1: [21834, 25805, 29415, 29054], 2: [], 3: []}
5 {0: 22, 1: 24, 2: 4, 3: 25} {0: [21840, 25811, 29421, 29060], 1: [451834521814, 30137, 545284, 2256424, 1303023, 3400072], 2: [], 3: []}
6 {0: 26, 1: 30, 2: 4, 3: 29} {0: [451834521820, 30143, 545290, 2256430, 1303029, 3400078, 552143], 1: [414963, 490412, 559002], 2: [], 3: []}
7 {0: 33, 1: 33, 2: 4, 3: 36} {0: [414969, 490418, 559008], 1: [8584855914583, 572720, 10360513, 42872173, 24757554, 64601485, 10490720], 2: [], 3: []}
8 {0: 36, 1: 40,

In [ ]:
inspected_items = [m.inspected_items for m in monkeys.values()]
inspected_items.sort()
inspected_items[-1]*inspected_items[-2]

In [ ]:
result = {k:m.inspected_items for k, m in monkeys.items()}
print(result)

In [ ]:
def represent_as_prime_numbers(number:int)->list[int]:
        prime_numbers = []
        if number > 1:
            for i in range(2, number//2):
                if (number % i) == 0:
                    prime_numbers.append(int(i))
                    number /= i
            else:
                prime_numbers.append(int(number))
        else:
            raise ValueError
    
        return prime_numbers

represent_as_prime_numbers(93)

In [ ]:
0%3